## Step 4 — Prepare Pretrained GoogLeNet Model

In this step, we:
1. Load the pretrained GoogLeNet model from `torchvision.models`
2. Replace its fully connected (FC) layer with a custom classifier
   suitable for the number of blood cancer cell classes in our dataset

> Note: At this stage, we are **preparing the model**, not yet training or fine-tuning.


In [3]:
#  Kaoutar — Load Pretrained GoogLeNet
import os
import torch
import torch.nn as nn
from torchvision import models


# Number of classes (adjust based on your dataset)
num_classes = len(os.listdir("../data/cleaned"))

# Load pretrained GoogLeNet
googlenet = models.googlenet(pretrained=True)

# Replace the fully connected (FC) layer
# GoogLeNet's FC layer is named 'fc'
googlenet.fc = nn.Sequential(
    nn.Linear(googlenet.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)

print(" GoogLeNet loaded and FC layer replaced.")
print(googlenet)


c:\Users\kaout\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\kaout\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to C:\Users\kaout/.cache\torch\hub\checkpoints\googlenet-1378be20.pth


100.0%


 GoogLeNet loaded and FC layer replaced.
GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, ep

##  Step 5 — Define Optimizer, Loss Function & Training Loop

In this step, we will:

1. Define the **loss function** suitable for multi-class classification (`CrossEntropyLoss`)  
2. Choose an **optimizer** (Adam or SGD) and learning rate  
3. Implement a **training loop** to train the model on the training set and evaluate on the validation set  

> Note: The model architecture is already prepared (GoogLeNet with custom FC layer).


In [ ]:
# === 🧩 Kaoutar — Training Setup ===


import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# --- Hyperparameters ---
learning_rate = 1e-4
batch_size = 16
num_epochs = 10  # adjust as needed

# --- Loss function ---
criterion = nn.CrossEntropyLoss()

# --- Optimizer ---
optimizer = optim.Adam(googlenet.parameters(), lr=learning_rate)
# Or use SGD if preferred:
# optimizer = optim.SGD(googlenet.parameters(), lr=learning_rate, momentum=0.9)

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
googlenet.to(device)

# --- DataLoaders ---
train_dataset = datasets.ImageFolder("../data/splits/train", transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
]))

val_dataset = datasets.ImageFolder("../data/splits/val", transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
]))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# --- Training Loop ---
for epoch in range(num_epochs):
    googlenet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = googlenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # --- Validation ---
    googlenet.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = googlenet(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
    
    val_loss /= len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {epoch_loss:.4f} "
          f"Val Loss: {val_loss:.4f} "
          f"Val Acc: {val_acc:.4f}")

print(" Training complete.")
